In [23]:
from PIL import Image
import random
import numpy as np
from fractions import Fraction

In [30]:
r = np.random.randint(0, 255, (10, 10))

array([[ 33,  89, 129,  15, 163,  45, 172,  16,  76,  68],
       [ 93, 118, 110, 119, 145, 222,  89, 203, 169, 251],
       [ 91, 140,  27,  63, 170, 209,  59,  35,  72,  72],
       [ 32,  45, 240, 151, 195,  97,  14,  29, 120, 196],
       [207, 153,  58, 250, 124, 201,  24, 128,  24, 193],
       [  2,   6, 217,  16, 104,  21,  44,  93,  92, 230],
       [ 69, 140, 212, 182,  92, 122, 250, 230,   3, 239],
       [ 43,  53, 104,  65,  75,   1, 162, 120, 159,  21],
       [ 39, 238, 158,  76,  78, 142, 209,  14,   4,  91],
       [203,  54, 131,  75,  10,  13,  48, 197, 108, 167]])

In [9]:
# сохраняем изображение
# img.save('random_color_pixels.png')

# покажем изображение
img.show()

## Задачи лабораторной работы №1

Демонстрируется результат каждой операции (до и после).  
1. Передискретизация (только для бакалавров).
    1) Растяжение (интерполяция) изображения в M раз;
    2) Сжатие (децимация) изображения в N раз;
    3) Передискретизация изображения в K=M/N раз путём растяжения и последующего сжатия (в два прохода);
    4) Передискретизация изображения в K раз за один проход.

#### 0. Создание изображения

In [58]:
# размеры изображения
width = 120
height = 90

In [132]:
def create_image(width, height):
    # создаем новое изображение
    img = Image.new('RGB', (width, height), color='white')
    # получаем доступ к пикселям изображения
    pixels = img.load()
    # создаем матрицу height * width * 3
    rgb = np.random.randint(0, 255, (height, width, 3))
    # проходим по каждому пикселю и задаем случайный цвет
    for i in range(width):
        for j in range(height):
            pixels[i, j] = (rgb[j, i, 0], rgb[j, i, 1], rgb[j, i, 2])
    return img

In [139]:
img = create_image(120,90)

In [140]:
img.show()

#### 1. Растяжение (интерполяция) изображения в M раз;

In [141]:
img.show()

In [142]:
def upsample_img(img, m):
    # получаем текущие размеры изображения
    width, height = img.size

    # создаем новое изображение с увеличенными размерами
    new_width = width * m
    new_height = height * m
    upsampled_img = Image.new('RGB', (new_width, new_height), color='white')

    # проходим по каждому пикселю в старом изображении и изменяем интенсивность
    for i in range(width):
        for j in range(height):
            # получаем цвет текущего пикселя
            r, g, b = upsampled_img.getpixel((i, j))

            # устанавливаем новый цвет пикселя в новом изображении
            for k in range(m):
                for l in range(m):
                    upsampled_img.putpixel((i*m + k, j*m + l), (r,g,b))
    return upsampled_img

In [145]:
upsampled_img = upsample_img(img, 3)
img.show()
upsampled_img.show()
print(f"Original image size: {img.size}; Upsampled image size: {upsampled_img.size}")

Original image size: (120, 90); Upsampled image size: (360, 270)


In [21]:
img.show()

##### 2. Сжатие (децимация) изображения в N раз;

1. Загрузка изображения и получение его размера (ширины и высоты).   
2. Вычисление нового размера изображения путем деления текущего размера на N.  
3. Создание пустой матрицы пикселей нового размера.  
4. Разбиение исходного изображения на блоки размером N x N пикселей.  
5. Вычисление среднего значения яркости пикселей в каждом блоке.  
6. Заполнение новой матрицы сжатыми блоками пикселей, используя среднее значение яркости пикселей каждого блока.  
7. Сохранение нового изображения.  

In [135]:
# img = create_image(1000, 500)

In [177]:
def downsample_img(img, N):
    # загрузка изображения и получение его размера
    width, height = img.size

    # вычисление нового размера изображения
    new_width = width // N
    new_height = height // N

    downsampled_img = Image.new('RGB', (new_width, new_height), color='white')
    # создание пустой матрицы пикселей нового размера
    new_pixels = downsampled_img.load()
    # разбиение исходного изображения на блоки размером N x N пикселей
    for i in range(new_height):
        for j in range(new_width):
            # вычисление среднего значения (r, g ,b) пикселей в каждом блоке
            x = i * N
            y = j * N
            block = np.array(img.crop((y, x, y + N, x + N)))
            red, green, blue = list(), list(), list()
            #print(block)
            for k in range(N):
                for m in range(N):
                    red.append(block[k, m, 0])
                    green.append(block[k, m, 1])
                    blue.append(block[k, m, 2])

            r = int(np.mean(red))
            g = int(np.mean(green))
            b = int(np.mean(blue))

            # заполнение новой матрицы сжатыми блоками пикселей
            new_pixels[j, i] = (r, g, b)

    #сохранение нового изображения
    # downsampled_img.save('compressed_img.png')
    return downsampled_img

In [178]:
downsampled_img = downsample_img(img, 3)
print(f"Original image size: {img.size}; Downsampled image size: {downsampled_img.size}")
img.show()
downsampled_img.show()

Original image size: (120, 90); Downsampled image size: (40, 30)


##### 3. Передискретизация изображения в K=M/N раз путём растяжения и последующего сжатия (в два прохода);

In [184]:
def resampling(img, k):
    frac = Fraction(k).limit_denominator()
    M = frac.numerator
    N = frac.denominator
    upsampled_img = upsample_img(img, M)
    downsampled_img = downsample_img(upsampled_img, N)
    return downsampled_img

In [185]:
img = create_image(120, 90)

In [191]:
resampled_img = resampling(img, 2.5)
img.show(title = 'original')
resampled_img.show('resampled')
print(f"Original image size: {img.size}; Downsampled image size: {resampled_img.size}")

Original image size: (120, 90); Downsampled image size: (300, 225)


#### 4. Передискретизация изображения в K раз за один проход.